# Семинар 3. Исправление опечаток

In [3]:
# !pip install razdel tqdm

In [1]:
import os, re
from string import punctuation
import numpy as np
import json
from collections import Counter
from pprint import pprint
from nltk import sent_tokenize
punctuation += "«»—…“”"
punct = set(punctuation)
from sklearn.metrics import classification_report, accuracy_score
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import numpy as np
from collections import Counter

In [2]:
# библиотека для отслеживания прогресса
from tqdm.notebook import tqdm

Возьмем данные с соревнования [Dialog Evaluation 2016](http://www.dialog-21.ru/evaluation/2016/spelling_correction/) по исправлению опечаток. Данные представляют собой набор предложений (правильное - ошибочное). Задача найти слова с ошибками и заменить их на правильный вариант.

Я удалили из данных случаи, когда в словах пропущен или вставлен пробел, чтобы было проще сопоставить слова в предложении. 

In [3]:
bad = open('data/sents_with_mistakes.txt', encoding='utf8').read().splitlines()
true = open('data/correct_sents.txt', encoding='utf8').read().splitlines()

In [4]:
len(true)

915

In [5]:
# Посмотрим на пары предложений
print(bad[2])
print(true[2])

Пояним эту мысль.
Поясним эту мысль


In [6]:
# напишем функцию, которая будет сопоставлять слова в правильном и ошибочном варианте
# разобьем предложение по пробелам и удалим пунктуация на границах слов
def align_words(sent_1, sent_2):
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()
    
    tokens_1 = [token.strip(punctuation) for token in tokens_1]
    tokens_2 = [token.strip(punctuation) for token in tokens_2]
    
    tokens_1 = [token for token in tokens_1 if token]
    tokens_2 = [token for token in tokens_2 if token]
    
    assert len(tokens_1) == len(tokens_2)
    
    return list(zip(tokens_1, tokens_2))

In [7]:
pprint(align_words(true[1], bad[1]))

[('апофеозом', 'опофеозом'),
 ('дня', 'дня'),
 ('для', 'для'),
 ('меня', 'меня'),
 ('сегодня', 'сегодня'),
 ('стала', 'стала'),
 ('фраза', 'фраза'),
 ('услышанная', 'услышанная'),
 ('в', 'в'),
 ('новостях', 'новостях')]


Вытащим только неправильные варианты и заодно посчитаем процент ошибок.

In [8]:
mistakes = []
total = 0
for i in range(len(true)):
    word_pairs = align_words(true[i], bad[i])
    
    
    for pair in word_pairs:
        if pair[0] != pair[1]:
            mistakes.append(pair)
        total += 1

In [9]:
print('Доля ошибок - ', len(mistakes)/total )

Доля ошибок -  0.12886443221610805


Обернем в Counter, чтобы сразу увидеть частотные ошибки.

In [10]:
Counter(mistakes).most_common(10)

[(('сегодня', 'седня'), 24),
 (('вообще', 'вобще'), 18),
 (('вообще', 'ваще'), 17),
 (('естественно', 'естесственно'), 17),
 (('хочется', 'хочеться'), 16),
 (('кстати', 'кстате'), 16),
 (('очень', 'ооочень'), 14),
 (('как-то', 'както'), 9),
 (('очень', 'оооочень'), 9),
 (('это', 'ето'), 9)]

Из-за того, что процент ошибок довольно низкий, не очень выгодно будет находить исправление для каждого слова. Нужен какой-то более простой классификатор, который выделит ошибочные слова, чтобы потом только их и редактировать.

Самый простой способ это сделать - составить словарь правильных слов и потом сравнивать с ним. Чтобы не делать этого вручную, можно взять какой-нибудь корпус текстов, прошедщих редактуру. Тексты из википедии для этого хорошо подходят.

In [11]:
corpus = open('data/wiki_data.txt', encoding='utf8').read()

Попробуем предсказать ошибку простым заглядыванием в словарь. Если слово не в словаре - оно неправильное.

In [12]:
# создаем словарь
vocab = Counter(re.findall('\w+', corpus.lower()))


In [13]:
def predict_mistaken(word, vocab):
    return 0 if word in vocab else 1

In [14]:
# для оценки создаем два списка y_true и y_pred
# проходимся по предложениям
# сопоставляем слова с помощью функции align_words
# проходимся по парам слов и
# если слова одинаковые добавляем в y_true 0 
# если слова разные добавляем в y_true 1
# предказываем ошибочность слова из bad списка 
# добавляем предсказание в список y_pred

y_true = []
y_pred = []

for i in range(len(true)):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        if pair[0] == pair[1]:
            y_true.append(0)
        else:
            y_true.append(1)
        
        y_pred.append(predict_mistaken(pair[1], vocab))
    

In [15]:
# оцените качество с помощью classification_report
print(classification_report(y_true, y_pred, ))

              precision    recall  f1-score   support

           0       0.98      0.91      0.94      8707
           1       0.59      0.88      0.71      1288

    accuracy                           0.91      9995
   macro avg       0.79      0.90      0.83      9995
weighted avg       0.93      0.91      0.91      9995



### Генерация исправлений

Теперь нужно думать о том, как исправить неправильные слова. Посмотрим как это можно делать на примере известного алгоритма Питера Норвига. Идея алгоритма очень простая - для каждого неправильного слова нужно сгенерировать варианты исправлений и выбрать из них тот , что есть в словаре, а если таких несколько, то выбрать наиболее вероятный. 

Неправильными считаются слова, которых нет в словаре (также как в функции выше). А вероятность слова расчитывается по формуле - абсолютная частота слова в корпусе разделить на количество слов в корпусе.

Абсолютные частоты лежат в счетчике

In [16]:
vocab.most_common(10)

[('в', 267296),
 ('и', 147115),
 ('на', 81926),
 ('с', 61681),
 ('года', 43894),
 ('по', 37235),
 ('году', 32197),
 ('из', 29150),
 ('был', 23293),
 ('не', 23228)]

В вероятности они преобразуются вот такой функцией

In [17]:
N = sum(vocab.values())

def P(word, N=N):
    return vocab[word] / N


Во втором подготовительном семинаре немного разбиралась работа с вероятностями в питоне (представление маленьких и больших чисел в питоне, свойства логарифмов вероятностей), будет полезно повторить - https://github.com/mannefedov/compling_nlp_hse_course/blob/master/notebooks/first_module_intro/02_ngrams.ipynb (раздел про вероятности в конце)

Теперь самое интересное - способ генерации вариантов исправлений. Они генерируются 4 типами эвристик: удаление, перестановка, замена, вставка. 

1) удаление - по очереди выбрасываем из слова 1 букву (слово - лово, сово, слво, слоо, слов)  
2) перестановка - по очереди меняем соседние буквы (слово - лсово, солво, слвоо, слоов)  
3) замена - по очереди заменям каждую букву на другую букву алфавита (слово - алово, блово, влово, глово...)  
4) вставка - по очереди вставляем между соседними буквами букву алфавита (слово - салово, сблово, свлово, сглово...)  

В алгоритма два уровня генерации - сначала генерируются варианты для оригинального слова, а потом варианты для каждого варианта. Таким образом, максимальное допустимое отличие для ошибки и предсказания - 2 буквы.

In [18]:
# оригинальный код вот тут - https://norvig.com/spell-correct.html
# я только адаптировал его под русский язык

def correction(word): 
    "Находим наиболее вероятное похожее слово"
    return max(candidates(word), key=P)

def candidates(word): 
    "Генерируем кандидатов на исправление"
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "Выбираем слова, которые есть в корпусе"
    return set(w for w in words if w in vocab)


def edits1(word):
    "Создаем кандидатов, которые отличаются на одну букву"
    letters    = 'йцукенгшщзхъфывапролджэячсмитьбюё'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "Создаем кандидатов, которые отличаются на две буквы"
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

Попробуем исправить

In [19]:
%%time
correction('сcолнце')

CPU times: user 231 µs, sys: 1 µs, total: 232 µs
Wall time: 233 µs


'солнце'

In [20]:
%%time
correction('ваще')

CPU times: user 206 µs, sys: 8 µs, total: 214 µs
Wall time: 240 µs


'чаще'

In [21]:
%%time
correction('опофеоз')

CPU times: user 233 µs, sys: 11 µs, total: 244 µs
Wall time: 245 µs


'апофеоз'

Выводов по единичным примерам не сделаешь, поэтому давайте запустим на всем нашем корпусе

Для оценки используем будем использовать три метрики:  
1) процент правильных слов;  
2) процент исправленных ошибок  
3) процент ошибочно исправленных правильных слов

In [22]:
correct = 0
total = 0

total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

cashed = {}
for i in tqdm(range(len(true))):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        # чтобы два раза не исправлять одно и тоже слово - закешируем его
        # перед тем как считать исправление проверим нет ли его в кеше
        
        predicted = cashed.get(pair[1], correction(pair[1]))
        cashed[pair[1]] = predicted
        
        
        if predicted == pair[0]:
            correct += 1
        total += 1
        
        if pair[0] == pair[1]:
            total_correct += 1
            if pair[0] !=  predicted:
                correct_broken += 1
        else:
            total_mistaken += 1
            if pair[0] == predicted:
                mistaken_fixed += 1


  0%|          | 0/915 [00:00<?, ?it/s]

Получается, что в целом не стало лучше. Хотя 50% опечаток исправляются корректно

In [23]:
print(correct/total)
print(mistaken_fixed/total_mistaken)
print(correct_broken/total_correct)

0.870735367683842
0.5108695652173914
0.07603077983231882


Ещё проблема тут в том, что алгоритм медленно работает для длинных слов.

In [24]:
%%time
correction('солнвце')

CPU times: user 330 µs, sys: 5 µs, total: 335 µs
Wall time: 447 µs


'солнце'

In [25]:
%%time
correction('насмехатьсяаававттававаываываы')

CPU times: user 1.22 s, sys: 7.24 ms, total: 1.23 s
Wall time: 1.24 s


'насмехатьсяаававттававаываываы'

Посмотрим, как исправляются самые частотные ошибки.

In [26]:
[(wt[0], wt[1], correction(wt[1])) for wt, _ in Counter(mistakes).most_common(10)]

[('сегодня', 'седня', 'седая'),
 ('вообще', 'вобще', 'вообще'),
 ('вообще', 'ваще', 'чаще'),
 ('естественно', 'естесственно', 'естественно'),
 ('хочется', 'хочеться', 'хочется'),
 ('кстати', 'кстате', 'кстати'),
 ('очень', 'ооочень', 'очень'),
 ('как-то', 'както', 'факто'),
 ('очень', 'оооочень', 'сорочень'),
 ('это', 'ето', 'что')]

### Метрики близости слов.

Вместо того, чтобы генерировать все варианты, можно искать похожие слова в словаре. Для этого нужно задать метрику похожести. Для исправления опечаток часто используются расстояния редактирования

Самое известное расстояние редактирования - расстояние Левенштейна. Тут мы не будет поднобно разбирать алгоритм, можете почитать [тут](https://ru.wikipedia.org/wiki/%D0%A0%D0%B0%D1%81%D1%81%D1%82%D0%BE%D1%8F%D0%BD%D0%B8%D0%B5_%D0%9B%D0%B5%D0%B2%D0%B5%D0%BD%D1%88%D1%82%D0%B5%D0%B9%D0%BD%D0%B0), посмотреть более понятный разбор [тут](https://www.youtube.com/watch?v=MiqoA-yF-0M), а код на питоне есть [тут](https://ru.wikibooks.org/wiki/%D0%A0%D0%B5%D0%B0%D0%BB%D0%B8%D0%B7%D0%B0%D1%86%D0%B8%D0%B8_%D0%B0%D0%BB%D0%B3%D0%BE%D1%80%D0%B8%D1%82%D0%BC%D0%BE%D0%B2/%D0%A0%D0%B0%D1%81%D1%81%D1%82%D0%BE%D1%8F%D0%BD%D0%B8%D0%B5_%D0%9B%D0%B5%D0%B2%D0%B5%D0%BD%D1%88%D1%82%D0%B5%D0%B9%D0%BD%D0%B0).
Про самого Левенштейна можно почитать вот тут - https://nplus1.ru/material/2017/09/25/vladimir-levenshtein

Основная идея - найти минимальное число исправлений, которое нужно сделать в слове А, чтобы получить слово Б. Причем допустимы только три вида исправлений - удаление, вставка, замена. 

Ещё есть расстояние Дамерау-Левенштейна - почти то же самое, только разрешена ещё операция перестановки.

Есть библиотека textdistance, в которой реализованы многие методы нахождения расстояний.

In [27]:
!pip install textdistance


[notice] A new release of pip available: 22.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [28]:
import textdistance

In [29]:
def get_closest_match_with_metric(text, lookup, topn=20, metric=textdistance.levenshtein):
    # Counter можно использовать и с не целыми числами
    similarities = Counter()
    
    for word in lookup:
        similarities[word] = metric.normalized_similarity(text, word) 
    
    return similarities.most_common(topn)

In [30]:
%%time
get_closest_match_with_metric('сонце', vocab, 20, textdistance.levenshtein)

CPU times: user 1.05 s, sys: 133 ms, total: 1.19 s
Wall time: 1.5 s


[('солнце', 0.8333333333333334),
 ('конце', 0.8),
 ('монце', 0.8),
 ('соне', 0.8),
 ('сонче', 0.8),
 ('донце', 0.8),
 ('солнцем', 0.7142857142857143),
 ('солнцев', 0.7142857142857143),
 ('солнца', 0.6666666666666667),
 ('сочное', 0.6666666666666667),
 ('синице', 0.6666666666666667),
 ('солнцу', 0.6666666666666667),
 ('сочные', 0.6666666666666667),
 ('свинце', 0.6666666666666667),
 ('сфорце', 0.6666666666666667),
 ('монцей', 0.6666666666666667),
 ('ньонце', 0.6666666666666667),
 ('соньер', 0.6666666666666667),
 ('сорное', 0.6666666666666667),
 ('донцем', 0.6666666666666667)]

In [31]:
%%time
get_closest_match_with_metric('сонце', vocab, 20, textdistance.damerau_levenshtein)

CPU times: user 1.04 s, sys: 9.96 ms, total: 1.05 s
Wall time: 1.05 s


[('солнце', 0.8333333333333334),
 ('конце', 0.8),
 ('монце', 0.8),
 ('соне', 0.8),
 ('сонче', 0.8),
 ('донце', 0.8),
 ('солнцем', 0.7142857142857143),
 ('солнцев', 0.7142857142857143),
 ('солнца', 0.6666666666666667),
 ('сочное', 0.6666666666666667),
 ('синице', 0.6666666666666667),
 ('солнцу', 0.6666666666666667),
 ('сочные', 0.6666666666666667),
 ('свинце', 0.6666666666666667),
 ('сфорце', 0.6666666666666667),
 ('монцей', 0.6666666666666667),
 ('ньонце', 0.6666666666666667),
 ('соньер', 0.6666666666666667),
 ('сорное', 0.6666666666666667),
 ('донцем', 0.6666666666666667)]

In [32]:
%%time
get_closest_match_with_metric('опофеоз', vocab, 5, textdistance.damerau_levenshtein)

CPU times: user 1.04 s, sys: 9.09 ms, total: 1.05 s
Wall time: 1.07 s


[('апофеоз', 0.8571428571428572),
 ('апофеоза', 0.75),
 ('апофеозом', 0.6666666666666667),
 ('апофеты', 0.5714285714285714),
 ('опорной', 0.5714285714285714)]

In [33]:
%%time
get_closest_match_with_metric('кул', vocab, 5, textdistance.damerau_levenshtein)

CPU times: user 1.05 s, sys: 10.4 ms, total: 1.06 s
Wall time: 1.07 s


[('кул', 1.0), ('акул', 0.75), ('коул', 0.75), ('куль', 0.75), ('кулл', 0.75)]

Можно немного ускорить поиск, сократив количество слов в словаре. Возьмем только те, что встречаются больше 5 раз.

In [34]:
vocab_top = {word:count for word, count in vocab.items() if count > 5}

In [35]:
%%time
get_closest_match_with_metric('сонце', vocab_top, 20, textdistance.damerau_levenshtein)

CPU times: user 211 ms, sys: 5.3 ms, total: 216 ms
Wall time: 229 ms


[('солнце', 0.8333333333333334),
 ('конце', 0.8),
 ('монце', 0.8),
 ('соне', 0.8),
 ('солнцем', 0.7142857142857143),
 ('солнца', 0.6666666666666667),
 ('солнцу', 0.6666666666666667),
 ('сочные', 0.6666666666666667),
 ('союзе', 0.6),
 ('зоне', 0.6),
 ('сносе', 0.6),
 ('фоне', 0.6),
 ('концу', 0.6),
 ('конца', 0.6),
 ('конец', 0.6),
 ('гонке', 0.6),
 ('донец', 0.6),
 ('сотне', 0.6),
 ('сон', 0.6),
 ('синие', 0.6)]

Сильно быстрее не стало

Еще в питоне есть встроенная библиотека для нахождения близких строк

In [36]:
from difflib import get_close_matches

In [37]:
%%time
get_close_matches('сонце', vocab_top.keys(), n=4)

CPU times: user 136 ms, sys: 2.7 ms, total: 139 ms
Wall time: 152 ms


['солнце', 'соне', 'солнцем', 'сотне']

Работает тоже не очень быстро.

С большим словарем даже оптимизированные версии будут работать очень долго. И принципиально с этим ничего не сделаешь, т.к. поиск минимального расстояния редактирования - сложный алгоритм, а хороший словарь должен быть большим. 

Но можно обратить внимание на то, что расстояние редактирование до большинства слов в словаре считать бессмысленно, т.к. они очевидно отличаются слишком сильно (например, слово "лес" и "заноза" настолько разные, что нам неважно какое между ними расстояние редактирования). Нам нужно придумать способ, который бы позволил нам выделить из словаря только те слова, до которых имеет смысл считать расстояние редактирования. 

Кажется, что **косинусное расстояние по мешку символов** хорошо для этого подходит. Близкими будут слова, состоящие из одинаковых символов. Расстояние редактирования между такими словами может быть и большим (акула-лука),  и маленьким (акула-акул), поэтому их придется проверить расстоянием левенштейна прежде, чем предсказывать. При этом мы можем быть уверены, что далекими по косинусу точно не будут слова с маленьким расстоянием редактирования! И соответственно их можно сразу отбросить из кандидатов на исправление.

Косинусное расстояние работает на векторах, а векторные операции сильно быстрее любых циклов (а внутри расстояния левенштейна иммено циклы).

Сделаем поиск похожих по векторам символов, из которых состоит слово. Косинусное расстояние между векторами слов не равно расстоянию редактирования, т.к. в нем не учитывается порядок символов. Близкими будут слова, состоящие из одинаковых символов. Но мы можем быть уверены, что близкими точно не будут слова с большим косинсным расстоянием, и можем их не проверять их честной метрикой редактирования.

In [38]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances

In [39]:
vocab = Counter(re.findall('\w+', corpus.lower()))

word2id = list(vocab.keys())
id2word = {i:word for i, word in enumerate(vocab)}


vec = CountVectorizer(analyzer='char', max_features=10000, ngram_range=(1,3))
X = vec.fit_transform(vocab)

In [40]:
def get_closest_match_vec(text, X, vec, topn=20):
    v = vec.transform([text])
    
    # вся эффективноть берется из того, что мы сразу считаем близость 
    # 1 вектора ко всей матрице (словам в словаре)
    # считать по отдельности циклом было бы дольше
    # вместо одного вектора может даже целая матрица
    # тогда считаться в итоге будет ещё быстрее
    
    similarities = cosine_distances(v, X)[0]
    topn = similarities.argsort()[:topn] 
    
    return [(id2word[top], similarities[top]) for top in topn]

In [41]:
%%time
get_closest_match_vec('сонце', X, vec) # это расстояние - чем меньше тем лучше

CPU times: user 230 ms, sys: 46.6 ms, total: 276 ms
Wall time: 315 ms


[('монце', 0.24999999999999978),
 ('конце', 0.24999999999999978),
 ('донце', 0.24999999999999978),
 ('херсонцев', 0.2640199278060127),
 ('саксонцев', 0.2640199278060127),
 ('сон', 0.2928932188134523),
 ('ньонце', 0.29985995798599496),
 ('олонце', 0.2998599579859951),
 ('соне', 0.3264246859454365),
 ('монцей', 0.3291796067500631),
 ('солнце', 0.3291796067500631),
 ('донцем', 0.3291796067500631),
 ('концессионное', 0.33217692887937156),
 ('концерн', 0.3545027756320972),
 ('монцезе', 0.3545027756320972),
 ('концессионном', 0.35948738477965136),
 ('концессионера', 0.36155760193093855),
 ('концессионером', 0.36555873142548445),
 ('лондонцев', 0.367544467966324),
 ('эстонцев', 0.37005921165128786)]

Напишем функцию, которая принимает слово и находит ближайшее к нему в словаре с помощью косинусного расстояния, а затем проверяет варианты честной метрикой редактирования. 

In [42]:
def get_closest_hybrid_match(text, X, vec, topn=3, metric=textdistance.damerau_levenshtein):
    candidates = get_closest_match_vec(text, X, vec, topn*4)
    lookup = [cand[0] for cand in candidates]
    closest = get_closest_match_with_metric(text, lookup, topn, metric=metric)

    
    return closest

In [43]:
%%time
get_closest_hybrid_match('сонце', X, vec)

CPU times: user 223 ms, sys: 26.4 ms, total: 249 ms
Wall time: 264 ms


[('солнце', 0.8333333333333334), ('монце', 0.8), ('конце', 0.8)]

Оценим такой метод исправления.

In [44]:
mistakes = []
total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

total = 0
correct = 0

cashed = {}
for i in tqdm(range(len(true))):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        if predict_mistaken(pair[1], vocab):
            pred = cashed.get(pair[1], get_closest_hybrid_match(pair[1], X, vec)[0][0])
            cashed[pair[1]] = pred
        else:
            pred = pair[1]
        
            
        if pred == pair[0]:
            correct += 1
        else:
            mistakes.append((pair[0], pair[1], pred))
        total += 1
            
        if pair[0] == pair[1]:
            total_correct += 1
            if pair[0] != pred:
                correct_broken += 1
        else:
            total_mistaken += 1
            if pair[0] == pred:
                mistaken_fixed += 1


  0%|          | 0/915 [00:00<?, ?it/s]

In [45]:
print(correct/total)
print(mistaken_fixed/total_mistaken)
print(correct_broken/total_correct)

0.8533266633316658
0.4704968944099379
0.09004249454461927


К сожалению лучше не стало. Но возможно дело в том, что в алгоритме Норвига мы учитывали вероятности слов, а тут нет. Помимо отдельных вероятностей давайте сразу добавим биграмов и будем считать вероятность всего готового предложения, чтобы выбирать наиболее вероятное продолжение. 

Посчитаем статистики

In [139]:
from nltk.tokenize import sent_tokenize
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize


def ngrammer(tokens, n=2):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams


def normalize(text):
    normalized_text = [word.strip(punctuation) for word in text.split()]
    normalized_text = [word.lower() for word in normalized_text if word]
    return normalized_text


In [140]:
corpus = open('data/wiki_data.txt', encoding='utf8').read()

In [141]:
sentences = [['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(corpus)]

In [142]:
unigrams = Counter()
bigrams = Counter()

for sentence in sentences:
    unigrams.update(sentence)
    bigrams.update(ngrammer(sentence))

In [143]:
def compute_sentence_proba(text, word_counts = unigrams, bigram_counts = bigrams):
    prob = 0
    tokens = normalize(text)
    for ngram in ngrammer(['<start>'] + tokens + ['<end>']):
        word1, word2 = ngram.split()
        if word1 in word_counts and ngram in bigram_counts:
            prob += np.log(bigram_counts[ngram]/word_counts[word1])
        else:
            prob += np.log(2e-5)
    
    return prob

In [144]:
compute_sentence_proba("some random text in english")

-66.32368277431473

Теперь давайте обернем пайплайн исправления в одну функцию, сначала без вероятностей

In [145]:
def correct_text(text):
    sentence = normalize(text)
    corrected_sentence = []
    for word in sentence:
        if predict_mistaken(word, vocab):
            pred = get_closest_hybrid_match(word, X, vec)[0][0]
            corrected_sentence.append(pred)
        else:
            corrected_sentence.append(word)
    
    return " ".join(corrected_sentence)
        
    


In [146]:
i = 2
(bad[i], correct_text(bad[i]))

('Пояним эту мысль.', 'эпоним эту мысль')

А теперь добавим учет нескольких вариантов на каждом шаге. В итоге с помощью функции itertools.product мы получим все возможные предложения с кандидатами, которые выдала наша модель

```
corrections = [[some, sum, sim], [can, cans], [say]]

itertools.product(corrections) ->
some can say 
sum can say 
sim can say 
some cans say
sum cans say 
sim cans say

In [ ]:
import itertools

def correct_text_with_lm(text):
    sentence = normalize(text)
    corrections = [] 
    
    for word in sentence:
        if predict_mistaken(word, vocab):
            preds = get_closest_hybrid_match(word, X, vec)
            preds = [p[0] for p in preds]
            corrections.append(preds)
        else:
            corrections.append([word])

    possible_sentences = [" ".join(words) for words in itertools.product(*corrections)]
    most_prob_sentence = max(possible_sentences, key=lambda x: compute_sentence_proba(x))
    
    return most_prob_sentence

In [148]:
i = 2
(bad[i], correct_text_with_lm(bad[i]))

('Пояним эту мысль.', 'понимая эту мысль')

In [ ]:
mistakes = []
total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

total = 0
correct = 0


for i in tqdm(range(len(true))):
    word_pairs = align_words(true[i], bad[i])
    corrected = correct_text(bad[i]).split()
    
    for i in range(len(word_pairs)):
        true_word = word_pairs[i][0]
        actual_word = word_pairs[i][1]
        pred = corrected[i]
            
        if pred == true_word:
            correct += 1
        else:
            mistakes.append((true_word, actual_word, pred))
        total += 1
            
        if true_word == actual_word:
            total_correct += 1
            if true_word != pred:
                correct_broken += 1
        else:
            total_mistaken += 1
            if true_word == pred:
                mistaken_fixed += 1


In [156]:
print(correct/total)
print(mistaken_fixed/total_mistaken)
print(correct_broken/total_correct)

0.8281481481481482
0.3888888888888889
0.10427350427350428


In [ ]:
mistakes = []
total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

total = 0
correct = 0


for i in tqdm(range(len(true))):
    word_pairs = align_words(true[i], bad[i])
    corrected = correct_text_with_lm(bad[i]).split()
    
    for i in range(len(word_pairs)):
        true_word = word_pairs[i][0]
        actual_word = word_pairs[i][1]
        pred = corrected[i]
            
        if pred == true_word:
            correct += 1
        else:
            mistakes.append((true_word, actual_word, pred))
        total += 1
            
        if true_word == actual_word:
            total_correct += 1
            if true_word != pred:
                correct_broken += 1
        else:
            total_mistaken += 1
            if true_word == pred:
                mistaken_fixed += 1


In [158]:
print(correct/total)
print(mistaken_fixed/total_mistaken)
print(correct_broken/total_correct)

0.8468529879572825
0.41578947368421054
0.08901070216653616


## Готовые инструменты

Есть несколько готовых опечаточников:  
1) Hunspell - https://pypi.org/project/hunspell/  
2) Jamspell - https://github.com/bakwc/JamSpell#python  
3) Яндекс.Спеллер - https://yandex.ru/dev/speller/ (только через API)


Если вам понадобится в серьезной задаче исправлять опечатки, то начните с них, а не с алгоритма Норвига.

Для Яндекс Спеллера есть питоновская библиотека, которая упрощает его использование. У него есть некоторые ограничения (10 к запросов в день), но для небольших проектов этого вполне достаточно.

In [52]:
!pip install pyaspeller

  Using cached pyaspeller-2.0.0-py3-none-any.whl (12 kB)

[notice] A new release of pip available: 22.3.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [55]:
# Так можно исправить предложение целиком
from pyaspeller import YandexSpeller
speller = YandexSpeller()
fixed = speller.spelled('Для Яндекс Спеллера есь питоновская библиатека, которай упрощает его использование. У него есть некоторые ограничения (10 к запросов в день), но для небольших проектов этого вполне достаточно.')
fixed

'Для Яндекс Спеллера есть питоновская библиотека, которая упрощает его использование. У него есть некоторые ограничения (10 к запросов в день), но для небольших проектов этого вполне достаточно.'

In [54]:
# А так проверить и исправить отдельное слово
from pyaspeller import Word
check = Word('приджлажение')
print(check.text, check.correct, check.spellsafe, check.variants)

/Users/mnefedov/.pyenv/versions/3.10.9/lib/python3.10/site-packages/pyaspeller/word.py:17: UserWarning: Class Word is deprecated. Use YandexSpeller().spelled(text) instead
  warnings.warn("Class Word is deprecated. Use YandexSpeller().spelled(text) instead")


приджлажение False предложение ['предложение', 'придлажение', 'приджложение']
